In [1]:
import polars as pl

In [2]:
def get_df(url):
    return pl.read_parquet(source=url)

In [3]:
def get_eleicao_18():
    return get_df(f"https://raw.githubusercontent.com/perferctstorm/DiscursoOdioEleicoes/main/Dados/Eleicoes/eleicao18_turno_01.parquet")

In [4]:
def get_municipios():
    return get_df(f"https://raw.githubusercontent.com/perferctstorm/DiscursoOdioEleicoes/main/Dados/Eleicoes/municipios.parquet") 

In [5]:
df_municipios = get_municipios()

In [6]:
df_municipios.limit(1)

codigo_tse,codigo_ibge,nome,uf,capital,latitude,longitude,ddd
i64,i64,str,str,i64,f64,f64,i64
93360,5200050,"""Abadia de Goiás""","""GO""",0,-16.7573,-49.4412,62


In [7]:
df_poll_18:pl.DataFrame = get_eleicao_18()
df_poll_18 = df_poll_18.filter(pl.col("CD_CARGO")==1)
df_poll_18 = df_poll_18.drop("NR_ZONA")
df_poll_18.limit(1)

ANO_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_MUNICIPIO,NM_MUNICIPIO,CD_CARGO,DS_CARGO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,QT_VOTOS_VALIDOS,SIGLA_2022,POSIC_IDEOLOGICO,SG_POSIC_IDEOLOGICO,SG_REGIAO,NM_REGIAO
i64,str,str,str,str,i64,str,i64,str,i64,str,str,i64,str,str,str,str,str
2018,"""F""","""SP""","""BR""","""BRASIL""",61948,"""ITAPIRAPUÃ PAULISTA""",1,"""Presidente""",45,"""PSDB""","""Partido da Social Democracia B…",394,"""PSDB""","""Centro Direita""","""CD""","""SE""","""Sudeste"""


In [8]:
cols_list:list[str] = df_poll_18.columns
cols_list.remove("QT_VOTOS_VALIDOS")

In [9]:
df_poll_18 = df_poll_18.group_by(cols_list).sum()

In [10]:
df_poll_18 = df_poll_18.with_columns(
    pl.col("QT_VOTOS_VALIDOS").sum().over("CD_MUNICIPIO").alias("TOTAL_VOTOS_MUNIC"),
    (pl.col("QT_VOTOS_VALIDOS")/ pl.col("QT_VOTOS_VALIDOS").sum().over("CD_MUNICIPIO")).alias("PCT_VOTOS_MUNIC"),
)

In [11]:
df_poll_18

ANO_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_MUNICIPIO,NM_MUNICIPIO,CD_CARGO,DS_CARGO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,SIGLA_2022,POSIC_IDEOLOGICO,SG_POSIC_IDEOLOGICO,SG_REGIAO,NM_REGIAO,QT_VOTOS_VALIDOS,TOTAL_VOTOS_MUNIC,PCT_VOTOS_MUNIC
i64,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,i64,i64,f64
2018,"""F""","""MG""","""BR""","""BRASIL""",46094,"""ILICÍNEA""",1,"""Presidente""",45,"""PSDB""","""Partido da Social Democracia B…","""PSDB""","""Centro Direita""","""CD""","""SE""","""Sudeste""",404,7050,0.057305
2018,"""F""","""MG""","""BR""","""BRASIL""",46256,"""IPATINGA""",1,"""Presidente""",15,"""MDB""","""Movimento Democrático Brasilei…","""MDB""","""Centro Direita""","""CD""","""SE""","""Sudeste""",992,128936,0.007694
2018,"""F""","""BA""","""BR""","""BRASIL""",35491,"""SANTA RITA DE CÁSSIA""",1,"""Presidente""",17,"""PSL""","""Partido Social Liberal""","""UNIÃO""","""Extrema Direita""","""ED""","""NE""","""Nordeste""",2386,14513,0.164404
2018,"""F""","""MG""","""BR""","""BRASIL""",48712,"""MORRO DA GARÇA""",1,"""Presidente""",15,"""MDB""","""Movimento Democrático Brasilei…","""MDB""","""Centro Direita""","""CD""","""SE""","""Sudeste""",32,1861,0.017195
2018,"""F""","""RS""","""BR""","""BRASIL""",86568,"""CHARRUA""",1,"""Presidente""",45,"""PSDB""","""Partido da Social Democracia B…","""PSDB""","""Centro Direita""","""CD""","""S""","""Sul""",97,1834,0.05289
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2018,"""F""","""GO""","""BR""","""BRASIL""",93718,"""GOIANÉSIA""",1,"""Presidente""",50,"""PSOL""","""Partido Socialismo e Liberdade""","""PSOL""","""Extrema Esquerda""","""EE""","""CO""","""Centro-Oeste""",114,36343,0.003137
2018,"""F""","""CE""","""BR""","""BRASIL""",14770,"""MORAÚJO""",1,"""Presidente""",30,"""NOVO""","""Partido Novo""","""NOVO""","""Direita""","""D""","""NE""","""Nordeste""",14,5540,0.002527
2018,"""F""","""RN""","""BR""","""BRASIL""",18473,"""SÃO MIGUEL""",1,"""Presidente""",18,"""REDE""","""Rede Sustentabilidade""","""REDE""","""Centro""","""C""","""NE""","""Nordeste""",24,11489,0.002089


In [12]:
df_poll_18.filter( (pl.col("NM_MUNICIPIO")=="FORTALEZA") & (pl.col("SG_UF")=="CE")).limit(1)

ANO_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_MUNICIPIO,NM_MUNICIPIO,CD_CARGO,DS_CARGO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,SIGLA_2022,POSIC_IDEOLOGICO,SG_POSIC_IDEOLOGICO,SG_REGIAO,NM_REGIAO,QT_VOTOS_VALIDOS,TOTAL_VOTOS_MUNIC,PCT_VOTOS_MUNIC
i64,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,i64,i64,f64
2018,"""F""","""CE""","""BR""","""BRASIL""",13897,"""FORTALEZA""",1,"""Presidente""",27,"""DC""","""Democracia Cristã""","""DC""","""Direita""","""D""","""NE""","""Nordeste""",515,1361788,0.000378


In [13]:
breaks = [.25, .4, .55]
domain=["<=25%", ">25%,<=40%", ">40%,<=55%",">55%"]

df_choropleth = df_poll_18.select(["CD_MUNICIPIO","TOTAL_VOTOS_MUNIC","QT_VOTOS_VALIDOS","PCT_VOTOS_MUNIC"])
df_choropleth = (
    df_choropleth
    .with_columns(
        pl.col("PCT_VOTOS_MUNIC").cut(breaks, labels=domain).alias("PCT_VOTOS_LIMIT")
    )
)
df_choropleth = (
    df_choropleth
    .join(df_municipios, left_on="CD_MUNICIPIO", right_on="codigo_tse", how="inner")
    .drop(pl.col(["capital","ddd"])) 
)

df_choropleth

CD_MUNICIPIO,TOTAL_VOTOS_MUNIC,QT_VOTOS_VALIDOS,PCT_VOTOS_MUNIC,PCT_VOTOS_LIMIT,codigo_ibge,nome,uf,latitude,longitude
i64,i64,i64,f64,cat,i64,str,str,f64,f64
46094,7050,404,0.057305,"""<=25%""",3130507,"""Ilicínea""","""MG""",-20.9402,-45.8308
46256,128936,992,0.007694,"""<=25%""",3131307,"""Ipatinga""","""MG""",-19.4703,-42.5476
35491,14513,2386,0.164404,"""<=25%""",2928406,"""Santa Rita de Cássia""","""BA""",-11.0063,-44.5255
48712,1861,32,0.017195,"""<=25%""",3143609,"""Morro da Garça""","""MG""",-18.5356,-44.601
86568,1834,97,0.05289,"""<=25%""",4305371,"""Charrua""","""RS""",-27.9493,-52.015
…,…,…,…,…,…,…,…,…,…
93718,36343,114,0.003137,"""<=25%""",5208608,"""Goianésia""","""GO""",-15.3118,-49.1162
14770,5540,14,0.002527,"""<=25%""",2308807,"""Moraújo""","""CE""",-3.46311,-40.6776
18473,11489,24,0.002089,"""<=25%""",2412500,"""São Miguel""","""RN""",-6.20283,-38.4947


In [18]:
from urllib.request import urlopen
import json
import plotly.express as px

with urlopen('https://raw.githubusercontent.com/perferctstorm/DiscursoOdioEleicoes/refs/heads/main/Dados/Eleicoes/geojs-100-mun_minifier_reduc.json') as response:
    municipios = json.load(response)

In [28]:
#fig = px.choropleth(df_choropleth, geojson=municipios, locations='codigo_ibge', color='PCT_VOTOS_LIMIT',
#                           color_continuous_scale="Viridis",
#                           range_color=(0, 4),
#                           #scope="usa",
#                           labels={'uf':'UF'}
#                          )

In [27]:
#import plotly.express as px

#df = px.data.election()
#geojson = px.data.election_geojson()

#fig = px.choropleth(df, geojson=geojson, color="Bergeron",
#                    locations="district", featureidkey="properties.district",
#                    projection="mercator"
#                   )
#fig.update_geos(fitbounds="locations", visible=False)
#fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
#fig.show()


In [ ]:
#


In [25]:
municipios["features"][0]["properties"]["id"]

'1100015'

In [20]:
df_choropleth = df_choropleth.to_pandas()

In [29]:
from urllib.request import urlopen
import json
import plotly.express as px

with urlopen('https://raw.githubusercontent.com/perferctstorm/DiscursoOdioEleicoes/refs/heads/main/Dados/Eleicoes/geojs-100-mun_minifier.json') as response:
    municipios = json.load(response)
    
import plotly.express as px
fig = px.choropleth(df_choropleth, geojson=municipios, color="PCT_VOTOS_LIMIT",
                    locations="codigo_ibge", featureidkey="properties.id",
                    projection="mercator"
                   )
fig.update_geos(fitbounds="locations", observed=True)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


C:\Users\s788865623\AppData\Local\anaconda3\lib\site-packages\plotly\express\_core.py:1958: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



ValueError: Invalid property specified for object of type plotly.graph_objs.layout.Geo: 'observed'

Did you mean "center"?

    Valid properties:
        bgcolor
            Set the background color of the map
        center
            :class:`plotly.graph_objects.layout.geo.Center`
            instance or dict with compatible properties
        coastlinecolor
            Sets the coastline color.
        coastlinewidth
            Sets the coastline stroke width (in px).
        countrycolor
            Sets line color of the country boundaries.
        countrywidth
            Sets line width (in px) of the country boundaries.
        domain
            :class:`plotly.graph_objects.layout.geo.Domain`
            instance or dict with compatible properties
        fitbounds
            Determines if this subplot's view settings are auto-
            computed to fit trace data. On scoped maps, setting
            `fitbounds` leads to `center.lon` and `center.lat`
            getting a

In [ ]:
df_choropleth[:1]